# 將 CODIS 下載回來的氣象觀測月報整理成一個 csv 檔

我們從中央氣象署 CODIS 網站 https://codis.cwa.gov.tw/StationData 下載各氣象站月報表，以及氣溫與風速的單項逐時月報表之後  
存放在 historical/data/weather/raw 裡面  
再經由這個筆記本整合  

Dependencies:  
pandas  
numpy  
tqdm

In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import datetime

In [2]:
station_id_table = {
    '臺北':'46692',
    '嘉義':'46748',
    '高雄':'46744',
    '東吉島':'46730',
    '臺中電廠':'C0FA4',
    '通霄漁港':'C0E92',
    '福寶':'C0G96',
    '臺西':'C0K53',
}

In [3]:
first_col_list = [
    '觀測時間(day)',
    '氣溫(℃)',
    '最高氣溫(℃)',
    '最低氣溫(℃)',
    '相對溼度(%)',
    '風速(m/s)',
    '風向(360degree)',
    '最大瞬間風(m/s)',
    '最大瞬間風風向(360degree)',
    '降水量(mm)',
    '降水時數(hour)',
    '日照時數(hour)',
    '日照率(%)',
    '全天空日射量(MJ/㎡)',
    '總雲量(0~10)'
]

## 初始參數

這邊定義整合的開始與結束月份、下載月報表的存放位置、整合後表格的存放位置，以及要不要覆寫原本的資料。

In [4]:
start_time = '2023-01'
end_time = '2024-09'
raw_data_path = '../data/weather/raw/'
raw_filenames = os.listdir(raw_data_path)
destination_filename = '../data/weather/finalized/big_table.csv'

# 要不要覆寫現有資料
renew = False
# 若目標檔案不存在，則 renew 設為 True
renew = renew or (not os.path.exists(destination_filename))

## 整合資料

In [5]:
def check_date_valid(start_time, filename):
    file_year = int(filename.split('-')[1])
    file_month = int(filename.split('-')[2].split('.')[0])
    start_year = int(start_time.split('-')[0])
    end_year = int(end_time.split('-')[0])
    start_month = int(start_time.split('-')[1])
    end_month = int(end_time.split('-')[1])
    if file_year * 100 + file_month < start_year * 100 + start_month:
        return False
    if file_year * 100 + file_month > end_year * 100 + end_month:
        return False
    return True

將月報表的每天資料整合進大表格

In [6]:
for station_name, station_id in station_id_table.items():
    this_fname_list = [raw_data_path + fn for fn in raw_filenames if station_id in fn and check_date_valid(start_time, fn) and not 'hour' in fn]
    print(station_name)
    for fn in this_fname_list:
        yr_month_str = fn.split('.')[-2][-7::].replace('-', '/') + '/'
        this_df = pd.read_csv(fn).loc[1::]
        this_col_name_list = [col for col in this_df.columns if col in first_col_list]
        this_df = this_df[this_col_name_list]
        this_df.insert(0, '站名', [station_name] * len(this_df))
        this_df['觀測時間(day)'] = [yr_month_str + s for s in this_df['觀測時間(day)']]
        this_df.rename({'觀測時間(day)':'日期'}, axis=1, inplace=True)

        if 'final_df' in globals():
            final_df = pd.concat([final_df, this_df], axis=0, ignore_index=True)
        else:
            final_df = this_df

# if not renew:
#     final_df.drop_duplicates(['站名', '日期'], ignore_index=True, keep='first', inplace=True)

臺北
嘉義
高雄
東吉島
臺中電廠
通霄漁港
福寶
臺西


## 資料轉換

處理非數字的表格值  
'T' 會出現在降水量欄位，代表有雨跡，但降水量少於 0.1mm ，所以這邊把 'T' 轉換成 0  
其他非數值符號都代表資料缺失，所以轉換成 Nan

In [7]:
for col in tqdm(final_df.columns):
    if not col in ['站名', '日期']:
        for i in final_df.index:
            try:
                final_df.loc[i, col] = float(final_df.loc[i, col])
            except:
                if final_df.loc[i, col] == 'T':
                    final_df.loc[i, col] = 0
                    continue
                final_df.loc[i, col] = np.nan

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:08<00:00,  1.83it/s]


In [8]:
final_df

,站名,日期,氣溫(℃),最高氣溫(℃),最低氣溫(℃),相對溼度(%),風速(m/s),風向(360degree),最大瞬間風(m/s),最大瞬間風風向(360degree),降水量(mm),降水時數(hour),日照時數(hour),日照率(%),全天空日射量(MJ/㎡),總雲量(0~10)
0,臺北,2023/01/01,19.3,22.0,16.8,83.0,3.2,90.0,12.5,100.0,1.5,2.6,0.5,4.7,8.98,9.0
1,臺北,2023/01/02,18.5,20.4,16.6,89.0,2.6,90.0,11.1,90.0,2.0,6.6,0.0,0.0,4.0,9.8
2,臺北,2023/01/03,16.1,17.7,14.4,89.0,3.0,90.0,12.6,120.0,0,2.3,0.0,0.0,7.1,10.0
3,臺北,2023/01/04,16.3,17.9,14.5,90.0,3.1,90.0,11.0,60.0,0,5.9,0.2,1.9,6.24,9.0
4,臺北,2023/01/05,19.3,22.2,17.2,90.0,2.0,90.0,9.3,90.0,1.0,2.8,1.3,12.3,9.25,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4925,臺西,2024/07/23,30.3,32.8,28.2,85.0,3.5,6.0,15.4,346.0,1.5,NaN,NaN,NaN,NaN,NaN
4926,臺西,2024/07/24,27.8,29.5,25.8,94.0,8.2,314.0,21.5,322.0,54.0,NaN,NaN,NaN,NaN,NaN
4927,臺西,2024/07/25,26.5,NaN,NaN,99.0,9.5,203.0,NaN,NaN,366.0,NaN,NaN,NaN,NaN,NaN
4928,臺西,2024/07/26,27.3,29.2,25.0,93.0,6.2,184.0,23.3,189.0,43.5,NaN,NaN,NaN,NaN,NaN


## 計算時間段平均風速與氣溫

i.e.:  
假設臺北站 2030 年 1 月 1 日各小時測得的氣溫如下：  
12:00  19.9  
13:00  20.1  
14:00  20.2  
15:00  19.8  
16:00  19.7  
17:00  19.0  
18:00  18.3  

執行下面函數的結果就會是
``` Python
avg_temp_df = get_avg_Temperature_within_hours('臺北', [12, 14], '午後平均氣溫')
print(avg_temp_df.loc['2030-01-01']['午後平均氣溫'])
# 輸出值為 19.9, 20.1, 20.2 的平均，為 20.66666666
```

In [9]:
def get_avg_Temperature_within_hours(station_name, hour_range, output_col_name):
    station_id = station_id_table[station_name]
    this_fname_list = [raw_data_path + fn for fn in raw_filenames if station_id in fn and check_date_valid(start_time, fn) and 'AirTemperature-hour' in fn]
    
    data_dict = {
        '站名': [],
        '日期': [],
        output_col_name: []
    }
    for fn in this_fname_list:
        yr_month_str = f"{fn.split('.')[-2].split('-')[1]}/{fn.split('.')[-2].split('-')[2]}/"
        this_df = pd.read_csv(fn)
        this_col_list = [s for s in this_df.columns if s.isdigit()]
        this_col_list = [s for s in this_col_list if int(s) >= hour_range[0] and int(s) <= hour_range[1]]
        
        for i in this_df.index:
            if this_df.loc[i]['日/時'] == '平均':
                continue
            temp_list = []
            for col in this_col_list:
                this_val = this_df.loc[i][col]
                if type(this_val) == str:
                    if not '.' in this_val:
                        continue
                temp_list.append(float(this_val))
            if yr_month_str == '2024/09':
                print(temp_list)
                
            date_str = yr_month_str + this_df.loc[i]['日/時']
            if len(temp_list) > 0:
                data_dict['站名'].append(station_name)
                data_dict['日期'].append(date_str)
                data_dict[output_col_name].append(np.round(np.mean(temp_list), decimals=1))

    return pd.DataFrame(data_dict)

In [10]:
def get_avg_WindSpeed_within_hours(station_name, hour_range, output_col_name):
    station_id = station_id_table[station_name]
    this_fname_list = [raw_data_path + fn for fn in raw_filenames if station_id in fn and check_date_valid(start_time, fn) and 'WindSpeed,WindDirection-hour' in fn]
    
    data_dict = {
        '站名': [],
        '日期': [],
        output_col_name: []
    }
    for fn in this_fname_list:
        yr_month_str = f"{fn.split('.')[-2].split('-')[1]}/{fn.split('.')[-2].split('-')[2]}/"
        this_df = pd.read_csv(fn)
        this_col_list = [s for s in this_df.columns if s.isdigit()]
        this_col_list = [s for s in this_col_list if int(s) >= hour_range[0] and int(s) <= hour_range[1]]
        
        for i in this_df.index:
            if this_df.loc[i]['日/時'] == '平均':
                continue
            temp_list = []
            for col in this_col_list:
                this_val = this_df.loc[i][col]
                if not len(this_val.split('/')) == 2:
                    continue
                if not '.' in this_val.split('/')[0]:
                    continue
                try:
                    temp_list.append(float(this_val.split('/')[0]))
                except:
                    print(this_val)
            if yr_month_str == '2024/09':
                print(temp_list)
                
            date_str = yr_month_str + this_df.loc[i]['日/時']
            if len(temp_list) > 0:
                data_dict['站名'].append(station_name)
                data_dict['日期'].append(date_str)
                data_dict[output_col_name].append(np.round(np.mean(temp_list), decimals=1))

    return pd.DataFrame(data_dict)

In [11]:
# 定義中文時間描述詞對應的時間區間
hour_description_dict = {
    '午後': [12, 14],
    '下午': [15, 17],
    '傍晚': [18, 20]
}

實際計算每天的午後、下午、傍晚的平均風速與氣溫

In [12]:
df_list_for_all = []
for station_name in station_id_table.keys():
    df_list_for_station = []
    for des in hour_description_dict.keys():
        wind_df = get_avg_WindSpeed_within_hours(station_name, hour_description_dict[des], f'{des}平均風速')
        temp_df = get_avg_Temperature_within_hours(station_name, hour_description_dict[des], f'{des}平均氣溫')
        this_df = pd.merge(wind_df, temp_df, on=['站名', '日期'], how='outer')
        df_list_for_station.append(this_df)

    for i, df in enumerate(df_list_for_station):
        if i == 0:
            df_station = df
        else:
            df_station = pd.merge(df_station, df, on=['站名', '日期'], how='outer')

    df_list_for_all.append(df_station)

hour_df = pd.concat(df_list_for_all, axis=0).reset_index(drop=True)
output_df = pd.merge(hour_df, final_df, on=['站名', '日期'], how='outer')

## 儲存結果

In [13]:
if not renew:
    old_df = pd.read_csv(destination_filename)
    output_df = pd.concat([old_df, output_df], axis=0)
    output_df.drop_duplicates(['站名', '日期'], ignore_index=True, keep='first', inplace=True)
    
output_df.to_csv(destination_filename, index=False, encoding='utf-8-sig')
display(output_df.head(5))

,站名,日期,午後平均風速,午後平均氣溫,下午平均風速,下午平均氣溫,傍晚平均風速,傍晚平均氣溫,氣溫(℃),最高氣溫(℃),...,風速(m/s),風向(360degree),最大瞬間風(m/s),最大瞬間風風向(360degree),降水量(mm),降水時數(hour),日照時數(hour),日照率(%),全天空日射量(MJ/㎡),總雲量(0~10)
0,臺北,2023/01/01,3.6,21.3,3.7,20.4,3.0,19.9,19.3,22.0,...,3.2,90.0,12.5,100.0,1.5,2.6,0.5,4.7,8.98,9.0
1,臺北,2023/01/02,2.7,19.9,3.1,19.1,3.2,17.7,18.5,20.4,...,2.6,90.0,11.1,90.0,2.0,6.6,0.0,0.0,4.0,9.8
2,臺北,2023/01/03,3.7,17.0,3.4,16.0,3.5,15.2,16.1,17.7,...,3.0,90.0,12.6,120.0,0.0,2.3,0.0,0.0,7.1,10.0
3,臺北,2023/01/04,4.1,16.9,3.1,16.9,3.2,17.0,16.3,17.9,...,3.1,90.0,11.0,60.0,0.0,5.9,0.2,1.9,6.24,9.0
4,臺北,2023/01/05,3.0,21.7,2.5,21.1,1.6,19.9,19.3,22.2,...,2.0,90.0,9.3,90.0,1.0,2.8,1.3,12.3,9.25,8.4
